In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # 使用cpu
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'  # 只显示 warning 和 Error
os.chdir("Molecule_VAE")  # 更改工作路径
import numpy as np
import pandas as pd

# rdkit stuff
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import Draw
import matplotlib.pyplot as plt

Traceback (most recent call last):
  File "d:\Code\chemical_vae-main\venv\lib\site-packages\rdkit\Chem\PandasTools.py", line 130, in <module>
    if 'display.width' in pd.core.config._registered_options:
AttributeError: module 'pandas.core' has no attribute 'config'


In [2]:
from chemvae.vae_utils import VAEUtils
from chemvae import mol_utils as mu

import keras
from keras.metrics import categorical_accuracy
import tensorflow as tf

Using TensorFlow backend.


In [9]:
args = {'load_from_dataset':False}
# args['dict'] = 'models/server/train_3'  # MP
args['dict'] = 'models/server/train_2'  # BP
# args['dict'] = 'models/server/train_5'  # PSA
# args['dict'] = 'models/server/train_6'  # viscosity
# args['dict'] = 'models/server/train_7'  # dielectric_constants

# Load a model
load_from_dataset = args['load_from_dataset']
vae = VAEUtils(directory=args['dict'])

d:\Code\chemical_vae-main\venv\lib\site-packages\keras\models.py:251: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Using standarized functions? True
Standarization: estimating mu and std values ...done!


In [ ]:
# Predict property
if load_from_dataset:
    csv_path = args['dict'] + '/val.csv'
    df = pd.read_csv(csv_path)
    i = 4  # 第几个
    smiles = df.iloc[i, 0]
else:
    smiles = "[Li+].[O-][N+]([O-])=O"  # 目前数据集不包含盐，盐的SMILES有"."
    # smiles = "OC(C(F)(F)F)C(F)(F)F"
    # smiles = "CC(O)C(C)O"
    # smiles = "O=Cc1ccccc1"

smiles_1 = mu.canon_smiles(smiles)  # 规范化smile表达
X_1 = vae.smiles_to_hot(smiles_1, canonize_smiles=False)
z_1 = vae.encode(X_1)
X_r = vae.decode(z_1)

print('{:20s} : {}'.format('Input', smiles_1))
print('{:20s} : {}'.format('Reconstruction', vae.hot_to_smiles(X_r, strip=True)[0]))

print('{:20s} : {}'.format('hot_key', X_1))
print('{:20s} : {}'.format('hot_key_Reconstruction', X_r))
loss = keras.backend.mean(keras.losses.categorical_crossentropy(tf.convert_to_tensor(X_1), tf.convert_to_tensor(X_r)))
sess = tf.Session()
print('{:20s} : {}'.format('categorical_crossentropy_loss', sess.run(loss)))

accuracy = keras.backend.mean(categorical_accuracy(tf.convert_to_tensor(X_1), tf.convert_to_tensor(X_r)))
print(sess.run(categorical_accuracy(tf.convert_to_tensor(X_1), tf.convert_to_tensor(X_r))))
print('{:20s} : {}'.format('categorical_accuracy', sess.run(accuracy)))

print('{:20s} : {} with norm {:.3f}'.format('Z representation', z_1.shape, np.linalg.norm(z_1)))

if load_from_dataset:
    df = pd.read_csv(args['dict']+"/train.csv")
    print('{:20s} : {}'.format('Properties', df.columns.tolist()[1:]))
    y_1 = vae.predict_prop_Z(z_1)[0]  # vae.predict_prop_Z()可以实现反归一化预测值
    y_p = df.iloc[i, 1:].values.tolist()
    print('{:20s} : {}'.format("Ground_truth:", y_p))
    print('{:20s} : {}'.format("Predict:", y_1))

    loss = tf.sqrt(keras.losses.mean_squared_error(tf.convert_to_tensor(np.array(y_1)), tf.convert_to_tensor(np.array(y_p))))
    print('{:20s} : {}'.format('RMSE', sess.run(loss)))

    normalize_out = args['dict'] + "/norm_data.csv"
    df = pd.read_csv(normalize_out)
    mean = df['mean']
    std = df['std']
    y_1 = ((y_1-mean)/std).values.tolist()
    y_p = ((y_p-mean)/std).values.tolist()

    loss = tf.sqrt(keras.losses.mean_squared_error(tf.convert_to_tensor(np.array(y_1)), tf.convert_to_tensor(np.array(y_p))))
    print('{:20s} : {}'.format('RMSE after norm', sess.run(loss)))

    relative_loss = tf.abs(tf.convert_to_tensor(np.array(y_1)) - tf.convert_to_tensor(np.array(y_p))) / tf.abs(tf.convert_to_tensor(np.array(y_1)))
    print('{:20s} : {}'.format('Relative loss', sess.run(relative_loss)))
